In [18]:
folder = "C:\Users\dipic\Google Drive\Gaming\Pen and Paper RPGs\Genesys\Adventure Design\\"
CreatureCatalogue = open(folder + "Creature Catalogue.txt","r")

for line_num,line_full in enumerate(CreatureCatalogue):
    #Replacing tabs with spaces, then squashing to single spaces makes transforming a little easier
    line = line_full.replace('\t', ' ')
    while line.find("  ") > -1:
        line = line.replace('  ', ' ')
    #I also want to remove trailing spaces from lines
    line = line.replace(' \n','\n')
        
    #if line != "\n":
        #print line_num
        #print line

In [17]:
#This works for rivals. I'll copy it for a nemesis just to make sure I don't break it

# I think the dudes with spells, and the complicated ones like the Dreadgazer, need to be done manually
folder = "C:\Users\dipic\Google Drive\Gaming\Pen and Paper RPGs\Genesys\Adventure Design\\"
#I will use this to keep track of all of the abilities with full text to see what I can replace
All_Abilities_List = []
All_Abilities_List_name_only = []
JSONtext = ''

char_list = ["Brawn","Agility","Intellect","Cunning","Willpower","Presence"]
    
CreatureCatalogue = open(folder + "Creature Catalogue.txt","r")

#Start at 999999 so when I use the initial between it's always false, but then after we set the ability
#line it works
ability_line_num = 999999

for line_num,line_full in enumerate(CreatureCatalogue):
    #print line_num
    line = line_full
    #Firstly, I want to replace all tabs and double-spaces with single spaces to make parsing easier
    line = line_full.replace('\t', ' ')
    while line.find("  ") > -1:
        line = line.replace('  ', ' ')
    #I also want to remove trailing spaces from lines
    line = line.replace(' \n','\n')
    
    
    #For the first line, we know it starts with the name, type, and description
    #print('Line Number: ' + str(line_num))
    #Print where I expect the characteristic text to be
    #print(line[:line.find(":")] in char_list)
    if line.find("(Minion)") > -1 or line.find("(Rival)") > -1 or line.find("(Nemesis)") > -1 or line.find("(Apex Nemesis)") > -1:
    #if line_num == 0:
        #Grab the character name
        JSONtext += '\t{\n\t\t"name": "'
        p1 = line.find(" (")
        JSONtext += line[:p1] + '",\n\t\t"type": "'

        #Grab the type
        p2 = line.find(")")
        adversary_type = line[p1+2:p2]
        JSONtext += adversary_type + '",\n\t\t"description": "'

        #Grab the description
        p1 = p2 + 3
        p2 = line.find("\n")
        JSONtext += line[p1+3:p2] + '",\n\t\t"characteristics": {' #\n\t\t\t"Brawn": '
        
    if line[:line.find(":")] in char_list:
        #Loop through characteristics
        for c in char_list:
            #print(line.find(c))
            #If we can find that characteristic on this line
            if line.find(c) > -1:
                p1 = line.find(c) + len(c) + 2
                #print(p1)
                if c == "Presence":
                    JSONtext += '\n\t\t\t"' + c + '": ' + line[p1:p1+1] + '\n\t\t},\n\t\t"derived": {'
                else:
                    #print('\n\t\t\t"' + c + '": ' + line[p1:p1+1] + ',')
                    #p2 = re.search('[0-9]', line)
                    JSONtext += '\n\t\t\t"' + c + '": ' + line[p1:p1+1] + ','
            
    if line[:line.find(":")] in ["Soak","Wounds", "Strain", "M/R Defense"]:
        #Loop through derived characteristics (for rivals at least)
        for d in ["Soak","Wounds", "Strain", "M/R Defense"]:
            if line.find(d) > -1:
                p1 = line.find(d) + len(d) + 2
                p2 = line.find(' ',p1)
                #I need to use [,] instead of /
                if d == "M/R Defense":
                    #I'm assuming the defence will always be single digits
                    #print('\n\t\t\t"defence": [' + line[p1:p1+1] + ',' + line[p1+2:p1+3] + ']')
                    JSONtext += '\n\t\t\t"defence": [' + line[p1:p1+1] + ',' + line[p1+2:p1+3] + ']\n\t\t},\n\t\t"skills": '
                else:
                    #print('\n\t\t\t"' + d.lower() + '": ' + line[p1:p2] + ',')
                    JSONtext += '\n\t\t\t"' + d.lower() + '": ' + line[p1:p2] + ','
                    
    #This is to make sure I start with the correct bracket
    if line[:line.find(":")] == "Skills":
        JSONtext += '{'
    if line[:line.find(":")] == "Skills (group only)":
        JSONtext += '['
    #Minion skills have "group only"
    #I need to handle Melee (X) skills and knowledge skills
    if line[:line.find(":")] == "Skills" or line[:line.find(":")] == "Skills (group only)":
        #Skills: Athletics 1, Brawl 2, Resilience 2, Melee (Light) 3
        SkillSplit = line.replace('\n','')[line.find(":")+2:].split(", ")
        for s in SkillSplit:
            #Minion skills have no number
            #print s
            #I need the replace so that Melee (Light) 2 becomes "Melee: Light": 2
            if line[:line.find(":")] == "Skills (group only)":
                JSONtext += '"' + s.replace(" (", ": ").replace(")","") + '",'
            else:
                #I'm assuming the skills will always be single digits
                JSONtext += '\n\t\t\t"' + s.replace(" (", ": ").replace(")","")[:-2] + '": ' + s.replace(" (", ": ").replace(")","")[-1:] + ","
        #Now take off that final comma
        #I should check here for talents, probably, but let's go one example at a time
        JSONtext = JSONtext[:len(JSONtext)-1]
        #This is to make sure I end with the correct bracket
        if line[:line.find(":")] == "Skills":
            JSONtext += '\n\t\t},\n\t\t"abilities": ['
        elif line[:line.find(":")] == "Skills (group only)":
            JSONtext += '],\n\t\t"abilities": ['
                  
    if line[:line.find(":")] == "Abilities":
        ability_line_num = line_num
        
    #This needs to handle the next few lines of abilities
    if line_num >= ability_line_num and line[:line.find(":")] != "Equipment" and line != "\n":
        #print line
        #I think this split is pointless as they are always on separate lines
        #AbilitySplit = line.replace('\n','').replace('Abilities: ', '').split(", ")
        Ability = line.replace('\n','').replace('Abilities: ', '')
        #print Ability
        #If the ability has no description in brackets, it must be a generic one
        if Ability.find("(") == -1:
            JSONtext += '"' + Ability + '", '
        else:
            #Keep track of all of the abilities to see what I can make generic
            All_Abilities_List.append(Ability)
            All_Abilities_List_name_only.append(Ability[:Ability.find("(")-1])
            JSONtext += '\n\t\t\t{"name": "' + Ability[:Ability.find("(")-1] + '", "description": "' + \
                            Ability[Ability.find("(")+1:Ability.find(")")] + '"}, '
        
    if line[:line.find(":")] == "Equipment":
        ability_line_num = 999999
        #Now take off that final comma and space from the abilities
        JSONtext = JSONtext[:len(JSONtext)-2] + '],\n\t\t"weapons": ['
        #WeaponSplit = line.replace('\n','')[line.find(":")+2:].split("), ")
        #Trying to keep the delimiters
        d = "), "
        #To make this delimiting easier
        newlinetext = line.replace('\n','')[line.find(":")+2:] + ", "
        #print newlinetext
        WeaponSplit =  [e+")" for e in newlinetext.split("), ")]
        for w in WeaponSplit:
            #First, make sure it's a weapon
            #print w
            if w.find("Damage") > -1:
                JSONtext += '\n\t\t\t{\n\t\t\t\t"name": "' + w[:w.find(" (")] + '",\n\t\t\t\t'
                WeaponStatSplit = w[w.find("(")+1:].split("; ")
                #First is skill
                JSONtext += '"skill": "' + WeaponStatSplit[0] + '",\n\t\t\t\t'
                #Then the damage
                JSONtext += '"damage": ' + WeaponStatSplit[1][WeaponStatSplit[1].find(" ")+1:] + ',\n\t\t\t\t'
                #Then the critical rating
                JSONtext += '"critical": ' + WeaponStatSplit[2][WeaponStatSplit[2].find(" ")+1:] + ',\n\t\t\t\t'
                #Then the range
                JSONtext += '"range": "' + WeaponStatSplit[3][WeaponStatSplit[3].find("[")+1:WeaponStatSplit[3].find("]")] + '",\n\t\t\t\t'
                #Now get the special qualities. Get rid of that trailing bracket
                Qualities = WeaponStatSplit[3][WeaponStatSplit[3].find("]")+3:].replace(')','').split(", ")
                JSONtext += '"qualities": ['
                for q in Qualities:
                    #print q
                    JSONtext += '"' + q + '",'
                #Now take off that final comma after the last quality
                JSONtext = JSONtext[:len(JSONtext)-1] + ']\n\t\t\t},'
                
        #Once all of the weapons are done, add the gear line and then loop through again
        #But first, take off the final comma from the last weapon
        #Somehow, only run this if there is an item of gear without damage...
        '''
        JSONtext = JSONtext[:len(JSONtext)-1] + '\n\t\t],\n\t\t"gear": ['
        for w in WeaponSplit:
            if w.find("Damage") == -1 and w.find("(") > -1:
                JSONtext += '"' + w + '",'
        '''
        #Now take off that final comma after the last item
        #Plus, add the tags. I won't try species or category tags yet, just source and CR
        #Worry about gear later
        #JSONtext = JSONtext[:len(JSONtext)-1] + '],\n\t\t"tags": [' + \
        JSONtext = JSONtext[:len(JSONtext)-1] + '\n\t\t],\n\t\t"tags": [' + \
                        '"source:Creature Catalogue", "Homebrew", "Fantasy", "Monster", ' + \
                        '"Combat CR:TBD", "Social CR:TBD", "General CR:TBD"]\n\t},\n'
    
#    if line_num >= 23:
#        break

from collections import Counter

print 'Duplicate abilities with exact text match:'
for a in Counter(All_Abilities_List):
    if Counter(All_Abilities_List)[a] > 1:
        print a
        
print 'Duplicate abilities with name match:'
for a in Counter(All_Abilities_List_name_only):
    if Counter(All_Abilities_List_name_only)[a] > 1:
        print a
        
#After going through all of that, remove the final ,
JSONtext = JSONtext[:-2]
#print JSONtext
CreatureCatalogue.close()

JSONformatFile = open(r"Creature Catalogue JSON.json","w+")
JSONformatFile.write(JSONtext)
JSONformatFile.close()

Duplicate abilities with exact text match:
Skitter (can move across walls, ceilings, and giant spider webs without penalty)
Teeming Threat (as a maneuver, a swarm may engage all living targets within Short range, and take an attack action against each target)
Night Vision (owlbears suffer no Perception penalties at night)
Swarm (halve the damage dealt to the swarm before applying soak, unless the weapon has the Blast or Burn quality [regardless of whether the quality is activated])
Flyer (Must spend one maneuver per turn to stay aloft)
Duplicate abilities with name match:
Wall Crawler
Amphibious
Darkvision
Swarm
Leap
Skitter
Voracious
Tunneling
Multiattack
Swoop Attack
Fearless
Flyer
Night Vision
Reel
False Appearance
Erupt
Teeming Threat
Pack Tactics
Regeneration
Berserker
Tremorsense
Anchored
